# The CAVEclient

The CAVEclient is a client side library to allow easy interaction with the services within CAVE (connectome annotation versioning engine, also known as Dynamic Annotation Framework), eg. the annotations, stateserver. The github repository is public:
https://github.com/seung-lab/CAVEclient

The library can be installed directly from the github repository or from the prebuilt versions using pip:
```
pip install caveclient
```


## Tutorials

This tutorial mainly covers the interactions with the materialized annotation tables. More information and better explanations of the other functionalities of the client can be found in the following tutorial. Please be advised that depending on your permission level you may not be able to execute all queries in this tutorial with the preset parameters as it was written with defaults for iarpa's microns project:
https://github.com/seung-lab/CAVEclient/blob/master/CAVEclientExamples.ipynb


## Authentication & Authorization

If this is your first time to interact with any part of CAVE, chances are you need to setup your local credentials for your FlyWire account first. Please follow the section "Setting up your credentials" at the beginning of the tutorial above to do so.

You will need to have access to the FlyWire's production dataset to retrieve annotations. Otherwise you will see

```HTTPError: 403 Client Error: FORBIDDEN for url```

errors upon querying the materialization server.

## Initializing the CAVEclient

The FrameworkClient is instantiated with a datastack name. A datastack is a set of segmentation, and annotation tables and lives within an aligned volume (the coordinate space). FlyWire's main datastack is `flywire_fafb_production`, the aligned volume is `fafb_seung_alignment_v0` (v14.1). For convenience, there are other defaults set on the datastack level.

In [1]:
import numpy as np
import datetime
import pandas as pd
from caveclient import CAVEclient

In [2]:
datastack_name = "flywire_fafb_production"
client = CAVEclient(datastack_name)

## Annotation tables

Annotations are represented by points in space and parameters (such as size, type). At specific timepoints, annotations are combined with the (proofread) segmentation to create a materialized version of the annotation table. The AnnotationEngine (`client.annotation`) owns the raw annotations and the Materialization Service (`client.materialize`) owns the materialized versions of these tables. 

To check what annotation tables are visible to you run

In [3]:
client.annotation.get_tables()

https://prod.flywire-daf.com/annotation/api/v2/aligned_volume/fafb_seung_alignment_v0/table


['synapses_nt_v1', 'nuclei_v1']

## Creating a table

All users with permissions to proofread in FlyWire can create annotation tables and upload annotations. Currently, only the user who created a table can upload annotations to it; all other users have read access to the table.\

Creating a table requires a unique `table_name`, a `schema`, a `description` and a `voxel_resolution`. The `voxel_resolution` defines the resolution with which annotations are uploaded. For instance, `[1, 1, 1]` would mean that annotations in this table will be uploaded in nanometer space. 

Schemas are managed in a separate [repository](https://github.com/seung-lab/EMAnnotationSchemas). Schemas can freely be chosen from [there](https://globalv1.daf-apis.com/schema/views/). If no applicable schema is available, we encourage users to create a new schema and submit a pull-request. 

In the example below, we are going to use the [`bound_tag`](https://globalv1.daf-apis.com/schema/views/type/bound_tag/view) schema which is an annotation with one coordinate and a text field. All annotations in a table follow the same schema.

To avoid the creation of a large list of test tables, the following code is embedded in markdown. 

```

table_name = "my_table"

description = """
This is a test table to demonstrate table creation and annotation upload.
The data in this table is random and should not be used for analysis.

This table was create by ..."""

client.annotation.create_table(table_name=random_table_name,
                               schema_name="bound_tag",
                               description=description,
                               voxel_resolution=[1, 1, 1])
                               
```

All tables are listed here: https://prod.flywire-daf.com/annotation/views/aligned_volume/fafb_seung_alignment_v0

A specific table can be viewed here: https://prod.flywire-daf.com/annotation/views/aligned_volume/fafb_seung_alignment_v0/table/table_name (with table_name replaced)

## Uploading and updating annotations

Next, we generate some annotations to be uploaded to the new table. Tables can be uploaded from pandas DataFrames with columns according to the schema. One can include an `id` column to specify specific annotation, otherwise `id`s are assigned by the backend. 

```
random_locations_nm = np.random.randint([448000, 185000,  87000], [588000, 292000,  90000], size=[100, 3], dtype=int)
random_tags = [f"tag {i}" for i in range(100)]

random_annotation_data = pd.DataFrame.from_dict({"pt_position": list(random_locations_nm), 
                                                 "tag": random_tags,
                                                 "id": np.arange(100, 200)})
```

Annotations can be uploaded using `client.annotation.post_annotation` or `client.annotation.post_annotation_df`.

```
client.annotation.post_annotation_df(table_name=random_table_name,
                                     df=random_annotation_data,
                                     position_columns=["pt_position"])
```

Updating annotations works similarly to uploading them in the first place. Updating requires `id`s to be defined.

```
random_locations_nm = np.random.randint([448000, 185000,  87000], [588000, 292000,  90000], size=[10, 3], dtype=int)
random_tags = [f"tag {i}" for i in range(10)]

random_annotation_data = pd.DataFrame.from_dict({"pt_position": list(random_locations_nm), 
                                                 "tag": random_tags,
                                                 "id": np.arange(100, 110)})
                                                 
client.annotation.update_annotation_df(table_name=random_table_name,
                                       df=random_annotation_data,
                                       position_columns=["pt_position"])
```

## Reading annotaitons

Annotations can be read directly from the annotaition service. However, no segment IDs will be assigned to the annotations through this interface. To access annotaions with segment IDs assigned, one must use the [materialization interface](https://github.com/seung-lab/CAVEclient/blob/master/FlyWireSynapseTutorial.ipynb).

Annotations can be read by ID. Here, we use the nucleus table (`nuclei_v1`, see it online [here](https://prod.flywire-daf.com/annotation/views/aligned_volume/fafb_seung_alignment_v0/table/nuclei_v1)) as example.

In [5]:
client.annotation.get_annotation(table_name="nuclei_v1",
                                 annotation_ids=[7416439, 7415718])

[{'volume': 6.65092096,
  'bb_start_position': [718496, 261792, 114640],
  'valid': True,
  'bb_end_position': [722016, 265120, 116200],
  'pt_position': [720000, 263360, 115520],
  'superceded_id': None,
  'created': '2021-06-23 19:55:35.166396',
  'deleted': 'None',
  'id': 7415718},
 {'volume': 11.52339968,
  'bb_start_position': [708800, 262112, 128200],
  'valid': True,
  'bb_end_position': [712064, 264832, 131080],
  'pt_position': [710592, 263392, 129800],
  'superceded_id': None,
  'created': '2021-06-23 19:55:35.160196',
  'deleted': 'None',
  'id': 7416439}]